In [ ]:
# Instalando dependências básicas
!pip install pandas streamlit requests python-dotenv

# Instalando bibliotecas para o LLM Open Source (Usaremos Transformers para carregar um modelo)
!pip install transformers accelerate bitsandbytes sentencepiece


In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from IPython.display import display, Markdown

# --- Seu código de SIMULAÇÃO DE DADOS (mantido para garantir a execução) ---

np.random.seed(42)
num_pedidos = 1000
estados = ["SP", "RJ", "MG", "BA", "PE", "RS", "PR", "AM", "GO"]

# Cálculo de probabilidade
pesos_produtos = np.linspace(0.01, 0.1, 20)
probabilidades_produtos = pesos_produtos / np.sum(pesos_produtos)

data = {
    'orderId': np.arange(1000, 1000 + num_pedidos),
    'preco_total_item': np.round(np.random.uniform(10.0, 500.0, num_pedidos), 2),
    'data_pedido': pd.to_datetime(pd.date_range(start='2024-01-01', periods=num_pedidos, freq='D')),
    'status_pedido': np.random.choice(["Completed", "Pending", "Canceled", "Processing"], num_pedidos, p=[0.75, 0.15, 0.05, 0.05]),
    'nome_cliente': [f"Cliente_{i % 100}" for i in range(num_pedidos)],
    'segmento_cliente': np.random.choice(["Gold", "Silver", "Bronze", "Standard"], num_pedidos, p=[0.15, 0.25, 0.35, 0.25]),
    'cidade': [f"Cidade_{i % 50}" for i in range(num_pedidos)],
    'estado': np.random.choice(estados, num_pedidos),
    'nome_produto': np.random.choice([f"Produto_{i}" for i in range(20)], num_pedidos, p=probabilidades_produtos),
    'quantidade': np.random.randint(1, 6, num_pedidos)
}
df = pd.DataFrame(data)

# Recriação das Colunas de Análise
def map_region(estado):
    if not estado: return "Desconhecida"
    estado = estado.upper()
    if estado in ["AC", "AP", "AM", "PA", "RO", "RR", "TO"]: return "Norte"
    elif estado in ["AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"]: return "Nordeste"
    elif estado in ["DF", "GO", "MT", "MS"]: return "Centro-Oeste"
    elif estado in ["ES", "MG", "RJ", "SP"]: return "Sudeste"
    elif estado in ["PR", "RS", "SC"]: return "Sul"
    else: return "Desconhecida"

df['regiao'] = df['estado'].apply(map_region)
df['mes_ano'] = df['data_pedido'].dt.to_period('M').astype(str)

# --- CRIAÇÃO E DEFINIÇÃO DO df_kpi ---
receita_total = df['preco_total_item'].sum()
pedidos_unicos = df['orderId'].nunique()
df_kpi = pd.DataFrame([{
    'receita_total': receita_total,
    'pedidos_unicos': pedidos_unicos,
    'ticket_medio': receita_total / pedidos_unicos if pedidos_unicos > 0 else 0
}])
# --------------------------------------------------------------------------------

# ⚡ TROCA DO MODELO PARA OTIMIZAÇÃO DA VELOCIDADE ⚡
MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"

# --- 1. Carregamento e Configuração do LLM ---

try:
    # Tenta usar as variáveis globais (se já carregadas)
    global tokenizer, model
    _ = tokenizer
    _ = model
    print(f"Modelo {MODEL_NAME} já estava carregado. Continuando...")

except NameError:
    # Se não estiverem carregadas, carrega o modelo Phi-3
    print(f"Carregando Modelo {MODEL_NAME}. Isso deve ser muito mais rápido que o Mistral 7B...")
    try:
        # Usamos torch_dtype=None, confiando no dispositivo automático para otimizar
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            device_map="auto",
            torch_dtype=torch.float16, # Ajuste para melhor desempenho na T4
            trust_remote_code=True # Necessário para o Phi-3
        )
        print(f"Modelo {MODEL_NAME} carregado com sucesso na GPU! Espere uma inferência muito mais rápida.")
    except Exception as e:
        print(f"Erro ao carregar o modelo {MODEL_NAME}. Verifique se o runtime está em GPU. Erro: {e}")
        raise SystemExit(e) # Para a execução se o modelo não carregar

# 🚀 FUNÇÃO DE GERAÇÃO DE INSIGHTS (Compatível com Phi-3 e limpeza) 🚀
def generate_insights(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]
    # O Phi-3 usa o template de chat do Mistral/Llama
    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = encodeds.to(model.device)

    generated_ids = model.generate(
        model_inputs,
        max_new_tokens=1000,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(generated_ids[0], skip_special_tokens=False)

    # Limpeza: Procurar pela tag de fechamento da instrução [/INST]
    try:
        start_index = response.index('[/INST]') + len('[/INST]')
        clean_response = response[start_index:].strip()
    except ValueError:
        clean_response = response.strip()

    return clean_response

# --- 2. Preparação dos Dados para o Prompt ---

data_summary = f"""
## Resumo do Dataframe de Vendas:
* **Total de Pedidos:** {df_kpi['pedidos_unicos'].iloc[0]}
* **Receita Total:** R$ {df_kpi['receita_total'].iloc[0]:,.2f}
* **Ticket Médio Global:** R$ {df_kpi['ticket_medio'].iloc[0]:,.2f}

## Amostra de Dados (df.head()):
{df.head().to_markdown(index=False)}

## Agregados Relevantes:
* **Receita por Status:**
{df.groupby('status_pedido')['preco_total_item'].sum().sort_values(ascending=False).to_markdown()}

* **Receita por Região (Top 3):**
{df.groupby('regiao')['preco_total_item'].sum().nlargest(3).to_markdown()}

* **Produtos Mais Vendidos (Top 5 em Receita):**
{df.groupby('nome_produto')['preco_total_item'].sum().nlargest(5).to_markdown()}

* **Segmento de Cliente (Receita Total):**
{df.groupby('segmento_cliente')['preco_total_item'].sum().sort_values(ascending=False).to_markdown()}
"""

# --- 3. Prompt Sênior para Geração de Insights (Otimizado para Português e Feedback Acionável) ---
analyst_prompt = f"""
Você é um Analista de Dados Sênior e fluente em Português do Brasil.
Sua tarefa é analisar os dados de vendas fornecidos abaixo e produzir uma análise focada em **Tomada de Decisão**.

Instruções Cruciais:
1.  **IDIOMA:** A resposta DEVE ser **100% em Português do Brasil**.
2.  **FORMATO:** Gere 5 insights essenciais, formatados como uma lista numerada.
3.  **ESTRUTURA:** Cada ponto deve seguir esta estrutura EXATA, utilizando negrito para destaque:
    * **Insight Chave:** [O fato principal e a evidência numérica (R$) extraída dos dados.]
    * **Recomendação Acionável:** [O que a equipe de vendas/marketing deve fazer com base no insight.]

4.  **REGRAS RÍGIDAS DE FORMATAÇÃO:**
    * NÃO use as palavras 'Analyze', 'Analysis', 'Recommendation' ou 'Recomendação' como cabeçalhos.
    * NÃO use marcadores de lista internos (como bullet points). Mantenha a resposta concisa.
    * Use o formato monetário (R$) corretamente.

--- DADOS PARA ANÁLISE ---
{data_summary}
--- FIM DOS DADOS ---

## 📊 Análise de 5 Pontos com Ações Recomendadas:
"""

# --- 4. Execução do Agente ---
print("\n" + "="*50)
print("🤖 AGENTE DE INSIGHTS DE IA EM EXECUÇÃO...")
print("="*50)

# Chamar a função de geração de insights
insights = generate_insights(analyst_prompt)

# --- 5. Exibição dos Resultados ---
print("\n" + "#"*50)
display(Markdown("## ✨ Insights Gerados pelo Modelo Phi-3 Mini"))
display(Markdown(insights))
print("#"*50)